# Entity Linking Pipeline for [DisTEMIST](https://temu.bsc.es/distemist/) Shared Task (10th BioASQ Workshop)

The goal is to link disease mentions in Spanish case reports to SNOMED CT IDs.

Highlights:
- Building a custom dictionary based on the (non-UMLS) DisTEMIST gazetteer + multilingual UMLS synonyms
- Using an Ensemble of TFIDFNgramLinker + SapBERTLinker
- Reranking with a trainable CrossEncoderReranker
- Evaluation and Error Analysis

In [1]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from pathlib import Path

In [3]:
from xmen.evaluation import entity_linking_error_analysis, evaluate_at_k
from xmen.linkers.util import filter_and_apply_threshold

**Note**: adapt `umls_meta_path` in `../conf/distemist.yaml` to point to the location of your UMLS release (you might want to set the environment variable `$UMLS_HOME`)

In [4]:
from xmen.confhelper import load_config
config = load_config('../conf/distemist.yaml')
base_path = Path(config.cache_dir)
tmp_path = Path('..') / 'temp' / 'distemist'

In [5]:
import datasets

ds = datasets.load_dataset('distemist', 'distemist_linking_bigbio_kb')

# Own validation set (20% of distemist training set / EL sub-track)
with open('distemist_validation_docs.txt', 'r') as fh:
    valid_ids = [l.strip() for l in fh.readlines()]
    
ds_train = ds['train'].filter(lambda d: d['document_id'] not in valid_ids)
ds_valid = ds['train'].filter(lambda d: d['document_id'] in valid_ids)

ds['train'] = ds_train
ds['validation'] = ds_valid

ds

Using the latest cached version of the module from /home/Florian.Borchert/.cache/huggingface/modules/datasets_modules/datasets/distemist/f63b2c6775932c342d7bff59d751d6139c0c52c5255f7fbb3e458d255728a8dd (last modified on Thu Apr 20 13:14:35 2023) since it couldn't be found locally at distemist., or remotely on the Hugging Face Hub.
Found cached dataset distemist (/home/Florian.Borchert/.cache/huggingface/datasets/distemist/distemist_linking_bigbio_kb/1.0.0/f63b2c6775932c342d7bff59d751d6139c0c52c5255f7fbb3e458d255728a8dd)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/Florian.Borchert/.cache/huggingface/datasets/distemist/distemist_linking_bigbio_kb/1.0.0/f63b2c6775932c342d7bff59d751d6139c0c52c5255f7fbb3e458d255728a8dd/cache-39869da4ad369074.arrow
Loading cached processed dataset at /home/Florian.Borchert/.cache/huggingface/datasets/distemist/distemist_linking_bigbio_kb/1.0.0/f63b2c6775932c342d7bff59d751d6139c0c52c5255f7fbb3e458d255728a8dd/cache-955d26b050eff51d.arrow


DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 466
    })
    test: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 250
    })
    validation: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 117
    })
})

# Preparation - Just run once

__Note__: This is a bit more involved, because we dynamically explore different combinations of DisTEMIST gazetteer and UMLS subsets.

In most normal cases, you just want to configure the target dictionary in your .yaml file and run `xmen dict` and `xmen index` with default settings.

## Download Distemist gazetteer

In [6]:
!mkdir -p ../temp; wget https://zenodo.org/record/6505583/files/dictionary_distemist.tsv?download=0 -O ../temp/dictionary_distemist.tsv

--2023-05-26 21:47:53--  https://zenodo.org/record/6505583/files/dictionary_distemist.tsv?download=0
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9885605 (9.4M) [application/octet-stream]
Saving to: ‘../temp/dictionary_distemist.tsv’

../temp/dictionary_ 100%[===================>]   9.43M  7.32MB/s    in 1.3s    

2023-05-26 21:47:56 (7.32 MB/s) - ‘../temp/dictionary_distemist.tsv’ saved [9885605/9885605]



## Setup dictionaries

Since the DisTEMIST Task only covers a subset of SNOMED CT codes, we retrieve only additional synonyms for the UMLS for these codes.
Thus, we need to map CUIs to SNOMED CT IDs and merge them with the DisTEMIST gazetteer.

In [7]:
dict_configs = ['distemist_gazetteer', 'distemist_umls_es', 'distemist_umls_en_es', 'distemist_umls_all'] 

In [8]:
from IPython.display import Markdown, display

# Run these commands from the root folder
for d in dict_configs:
    display(Markdown(f"`xmen dict conf/distemist.yaml --code dicts/distemist.py --key {d}`"))

`xmen dict conf/distemist.yaml --code dicts/distemist.py --key distemist_gazetteer`

`xmen dict conf/distemist.yaml --code dicts/distemist.py --key distemist_umls_es`

`xmen dict conf/distemist.yaml --code dicts/distemist.py --key distemist_umls_en_es`

`xmen dict conf/distemist.yaml --code dicts/distemist.py --key distemist_umls_all`

In [9]:
meta_path = Path(config.umls_meta_path)

In [10]:
from collections import defaultdict
from tqdm.auto import tqdm
from xmen.umls import read_umls_file_headers

def read_cui2snomed_mapping(meta_path):
    mrconso = 'MRCONSO.RRF'
    cui2snomed = defaultdict(list)
    headers = read_umls_file_headers(meta_path, mrconso)
    with open(f"{meta_path}/{mrconso}") as fin:
        for line in tqdm(fin.readlines()):
            splits = line.strip().split("|")
            assert len(headers) == len(splits)
            concept = dict(zip(headers, splits))
            if concept['SAB'] in ['SNOMEDCT_US', 'SCTSPA']:
                cui2snomed[concept['CUI']].append(concept['SCUI'])
    return cui2snomed

In [11]:
cui2snomed_mapping = read_cui2snomed_mapping(meta_path)

  0%|          | 0/16857345 [00:00<?, ?it/s]

In [12]:
import json
from tqdm.auto import tqdm
from xmen.kb import CompositeKnowledgebase

def cui2snomed(entry):
    res = []
    cui = entry['concept_id']
    for sctid in cui2snomed_mapping[cui]:
        r = entry.copy()
        r['concept_id'] = sctid
        res.append(r)
    return res

def merge_and_write_dicts(target_jsonl, added_jsonl, output_jsonl):
    print(f'Writing to {output_jsonl}')
    output_jsonl.parent.mkdir(exist_ok=True, parents=True)
    cui_count = alias_count = 0
    kb = CompositeKnowledgebase([added_jsonl], mappers=[cui2snomed])
    with open(output_jsonl, 'w') as fo:
        for l in tqdm(list(open(target_jsonl).readlines())):
            cui_count += 1
            entry = json.loads(l)
            sctid = str(entry['concept_id'])
            if sctid in kb.cui_to_entity:
                concept = kb.cui_to_entity[sctid]
                known_aliases = [entry['canonical_name']] + entry['aliases']
                new_aliases = [c for c in [concept.canonical_name] + concept.aliases if c not in known_aliases]
            else:
                new_aliases = []
            entry['aliases'] += new_aliases
            alias_count += (len(entry['aliases']) + 1)
            fo.write(json.dumps(entry) + '\n')
    print(f'Written {cui_count} concepts with {alias_count} aliases')

In [13]:
distemist_gazetteer_jsonl = base_path / 'distemist_gazetteer' / 'distemist_gazetteer.jsonl'

In [14]:
for d in dict_configs[1:]:
    merge_and_write_dicts(distemist_gazetteer_jsonl, base_path / d / f'{d}.jsonl', base_path / 'distemist' / 'merged' / f'{d}.jsonl')

Writing to /home/Florian.Borchert/.cache/xmen/distemist/merged/distemist_umls_es.jsonl


  0%|          | 0/111179 [00:00<?, ?it/s]

Written 111179 concepts with 495324 aliases
Writing to /home/Florian.Borchert/.cache/xmen/distemist/merged/distemist_umls_en_es.jsonl


  0%|          | 0/111179 [00:00<?, ?it/s]

Written 111179 concepts with 1520890 aliases
Writing to /home/Florian.Borchert/.cache/xmen/distemist/merged/distemist_umls_all.jsonl


  0%|          | 0/111179 [00:00<?, ?it/s]

Written 111179 concepts with 2482643 aliases


## Prepare TF-IDF NGram and SapBERT indices for all configurations

In [16]:
# Run these commands from the root folder
display(Markdown(f"`xmen index conf/distemist.yaml --dict {base_path / 'distemist_gazetteer' / 'distemist_gazetteer.jsonl'} --output {base_path / 'distemist' / 'merged' / 'distemist_gazetteer'} --all`"))
for d in dict_configs[1:]:
    display(Markdown(f"`xmen index conf/distemist.yaml --dict {base_path / 'distemist' / 'merged' / f'{d}.jsonl'} --output {base_path / 'distemist' / 'merged' / f'{d}'} --all`"))

`xmen index conf/distemist.yaml --dict /home/Florian.Borchert/.cache/xmen/distemist_gazetteer/distemist_gazetteer.jsonl --output /home/Florian.Borchert/.cache/xmen/distemist/merged/distemist_gazetteer --all`

`xmen index conf/distemist.yaml --dict /home/Florian.Borchert/.cache/xmen/distemist/merged/distemist_umls_es.jsonl --output /home/Florian.Borchert/.cache/xmen/distemist/merged/distemist_umls_es --all`

`xmen index conf/distemist.yaml --dict /home/Florian.Borchert/.cache/xmen/distemist/merged/distemist_umls_en_es.jsonl --output /home/Florian.Borchert/.cache/xmen/distemist/merged/distemist_umls_en_es --all`

`xmen index conf/distemist.yaml --dict /home/Florian.Borchert/.cache/xmen/distemist/merged/distemist_umls_all.jsonl --output /home/Florian.Borchert/.cache/xmen/distemist/merged/distemist_umls_all --all`

# Candidate Generation

We configure two candidate generators:
1. `TFIDFNGramLinker`, which links surface forms based on TF-IDF scores over character ngrams
2. `SapBERTLinker`, which uses a Transformer model to retrieve candidates with dense embeddings

Finally, we combine their predictions using an `EnsembleLinker`

In [ ]:
dict_config = 'distemist_umls_en_es'

In [ ]:
from xmen.linkers import TFIDFNGramLinker, SapBERTLinker, EnsembleLinker
from xmen.linkers.util import filter_and_apply_threshold
from datasets import DatasetDict

In [ ]:
index_base_path = base_path / 'distemist' / 'merged' / dict_config / 'index'

#### TF-IDF over character n-grams

In [ ]:
ngram_linker = TFIDFNGramLinker(index_base_path=index_base_path / 'ngrams', k=100)
pred_ngram = ngram_linker.predict_batch(ds)

In [ ]:
# Recall for different numbers of candidates (k)
_ = evaluate_at_k(ds['validation'], pred_ngram['validation'])

#### Dense Retrieval with SapBERT

In [ ]:
# Clear singleton to free up memory
SapBERTLinker.clear()
sapbert_linker = SapBERTLinker(
    index_base_path = index_base_path / 'sapbert',
    k = 1000
)

pred_sapbert = sapbert_linker.predict_batch(ds, batch_size=128)

In [ ]:
# Recall for different numbers of candidates (k)
_ = evaluate_at_k(ds['validation'], pred_sapbert['validation'])

#### Ensemble

In [ ]:
ensemble_linker = EnsembleLinker()
ensemble_linker.add_linker('sapbert', sapbert_linker, k=100)
ensemble_linker.add_linker('ngram', ngram_linker, k=100)

# Re-use predictions for efficiency
# TODO: reuse_preds currently does not work with dataset dicts
pred_ensemble = DatasetDict()
pred_ensemble['train'] = ensemble_linker.predict_batch(ds['train'], 128, 100, reuse_preds={'sapbert' : pred_sapbert['train'], 'ngram' : pred_ngram['train']})
pred_ensemble['validation'] = ensemble_linker.predict_batch(ds['validation'], 128, 100, reuse_preds={'sapbert' : pred_sapbert['validation'], 'ngram' : pred_ngram['validation']})
pred_ensemble['test'] = ensemble_linker.predict_batch(ds['test'], 128, 100, reuse_preds={'sapbert' : pred_sapbert['test'], 'ngram' : pred_ngram['test']})

pred_ensemble.save_to_disk(tmp_path / dict_config / 'pred_ensemble')

In [ ]:
# Recall for different numbers of candidates (k)
_ = evaluate_at_k(ds['validation'], pred_ensemble['validation'])

# Reranking

We use a cross-encoder to embed the mention with their context together with all potential candidates. This way, we can learn the best ranking of candidates from the training data

In [ ]:
from xmen.reranking.cross_encoder import CrossEncoderReranker, CrossEncoderTrainingArgs

In [ ]:
K_RERANKING = 64
CROSS_ENC_MODEL = 'PlanTL-GOB-ES/roberta-base-biomedical-clinical-es'
NUM_EPOCHS = 20

## Prepare Data for Cross-Encoder Training
Create a dataset of samples for each instance with all potential candidates

In [ ]:
# Dictionary used for synonyms in entity representation. 
# Note: You may or may not use the config used for candidate generation here. We used a different one, to demonstrate the available options.
dict_config_rr = 'distemist_umls_es'

Use the ensemble predictions as candidates

In [ ]:
from xmen.linkers.util import filter_and_apply_threshold

candidates = datasets.load_from_disk(tmp_path / dict_config / 'pred_ensemble')
candidates = filter_and_apply_threshold(candidates, K_RERANKING, 0.0)

We load a knowledgebase to retrieve the synonyms per concept

In [ ]:
from xmen.kb import load_kb
kb = load_kb(base_path / 'distemist' / 'merged' / f'{dict_config_rr}.jsonl')

In [ ]:
cross_enc_ds = CrossEncoderReranker.prepare_data(candidates, ds, kb, encode_sem_type=True)
cross_enc_ds.save_to_disk(tmp_path / f'cross_enc_ds_{dict_config_rr}')

## Train Cross Encoder

The cross-encoder is trained for 20 epochs, the final model is saved to `output_dir`

In [ ]:
from xmen.data.indexed_dataset import IndexedDatasetDict
cross_enc_ds = IndexedDatasetDict.load_from_disk(tmp_path / f'cross_enc_ds_{dict_config_rr}')
cross_enc_ds

In [ ]:
cross_enc_ds['train'].dataset[0][0:10]

In [ ]:
train_args = CrossEncoderTrainingArgs(
    CROSS_ENC_MODEL, 
    NUM_EPOCHS,
    score_regularization=True,
)

In [ ]:
rr = CrossEncoderReranker()
output_dir = f'{tmp_path}/cross_encoder_training/'

In [ ]:
# Depending on the number of epochs, this can take a few hours
rr.fit(cross_enc_ds['train'].dataset, cross_enc_ds['validation'].dataset, output_dir=output_dir, training_args=train_args)

## Predict on Test Set

We load the final model to get predictions on the test set and run the evaluation

In [ ]:
rr = CrossEncoderReranker.load(output_dir, device=0)

In [ ]:
cross_enc_pred_valid = rr.rerank_batch(candidates['validation'], cross_enc_ds['validation'])

In [ ]:
_ = evaluate_at_k(ds['validation'], cross_enc_pred_valid)

In [ ]:
cross_enc_pred = rr.rerank_batch(candidates['test'], cross_enc_ds['test'])

In [ ]:
_ = evaluate_at_k(ds['test'], cross_enc_pred)

# Error Analysis

Using the `xmen.evaluation` module, we gain insights into different error types

In [ ]:
from notebook_util import analyze
from xmen.kb import load_kb
from xmen.evaluation import entity_linking_error_analysis

In [ ]:
cg_kb = load_kb(base_path / 'distemist' / 'merged' / f'{dict_config}.jsonl')

In [ ]:
error_cand = entity_linking_error_analysis(ds['validation'], candidates['validation'])
edf_cand, _ = analyze(error_cand, cg_kb, 'eval', None)

In [ ]:
# False negative (not part of top 64)
fns_cand = edf_cand[edf_cand.pred_index == -1]
fns_cand_counts = fns_cand.error_type.value_counts() / len(edf_cand)
fns_cand_counts

In [ ]:
# Ranking errors during candidate generation
misranked_cand = edf_cand[edf_cand.pred_index > 0]
misranked_cand_counts = misranked_cand.error_type.value_counts() / len(edf_cand)
misranked_cand_counts

In [ ]:
# Ranking errors after reranking
error_rr = entity_linking_error_analysis(ds['validation'], cross_enc_pred_valid)
edf_rr, _ = analyze(error_rr, cg_kb, 'eval', None)

misranked_rr = edf_rr[edf_rr.pred_index > 0]
misranked_rr_counts = misranked_rr.error_type.value_counts() / len(edf_rr)
misranked_rr_counts

In [ ]:
print('Recall@1', (edf_rr.pred_index == 0).sum() / len(edf_rr))
print('Recall@64', (edf_rr.pred_index >= 0).sum() / len(edf_rr))

In [ ]:
import matplotlib
from matplotlib import pyplot as plt

error_types = ['Complex Entity', 'Abbreviation', 'Wrong Type', 'Homonyms', 'Other Error']
keys = ['COMPLEX_ENTITY', 'ABBREV', 'WRONG_SEMANTIC_TYPE', 'SAME_SYNONYMS', 'UNKNOWN_ERROR']
counts = [fns_cand_counts, misranked_cand_counts, misranked_rr_counts]
count_names = ['False Negatives', 'Candidate Generation Errors', 'Reranking Errors']

hatches = ['', '\\\\\\', '///']
colors = ['lightyellow', '#C0F5EC', '#118470']

matplotlib.rcParams['hatch.linewidth'] = 0.3 
matplotlib.rcParams.update({'font.size': 12})

fig, axs = plt.subplots(1, len(keys), figsize=(8, 2))
for i, (key, error) in enumerate(zip(keys, error_types)):
    for j, bar in enumerate(axs[i].bar(x=count_names, height=[c.get(key, 0.0) for c in counts], color=colors, edgecolor = 'black', linewidth=0.2)):
        bar.set_hatch(hatches[j])
    axs[i].grid(axis='y')
    axs[i].set_ylim([0.0, 0.2])
    axs[i].set_xlabel(error, labelpad=10)
    axs[i].get_xaxis().set_ticks([])
handles = [plt.Rectangle((0,0),1,1, facecolor=color, linewidth=0.2, edgecolor='black', hatch=h) for color, h in zip(colors, hatches)]
plt.legend(handles, count_names, loc='upper center', bbox_to_anchor=(-3.1, 1.4, 0, 0), ncol=len(counts))

plt.subplots_adjust(wspace=0.7)